# STag Demonstration

The following iPython Jupyter notebook gives a step-by-step demonstration of how to use STag to get the tag probabilities and the predicted class for a spectra.

# Setup

The first step is to read in the beta values for each of the tags as well as an example spectrum (this can be modified to read in an appropriate spectrum of your choice).

In [ ]:
import beta_reader

beta = beta_reader.beta_reader()
spectra = np.load('/aty.npy')
names = 'DES15C2aty'
z = 0.149

OSError: ignored

# Pre-processing

In order to use STag, spectra need to be pre-processed appropriately. This involves filtering, de-redshifting, binning, continuum removal, apodisation, and scaling.

All of these steps are handled by the spectra_preprocessing package, which largely uses methods made for the software [DASH](https://github.com/daniel-muthukrishna/astrodash).

In [ ]:
import spectra_preprocessing as sp

#Read in the fits file of the spectra and extract the flux and wavelength
fits_file = spectra
table = fits.open(fits_file)
flux = table[0].data
w0 = table[0].header['CRVAL1']
dw = table[0].header['CDELT1']
p0 = table[0].header['CRPIX1']
nlam = len(flux)
wave = w0+dw*(np.arange(nlam, dtype='d')-p0)
table.close()

full = np.column_stack((wave, flux))

#Initialise for pre-processing
preProcess = sp.PreProcessing(full, 2500, 10000, 1024)

#Do the pre-processing steps
sfWave, sfFlux, minInd, maxInd, sfZ, sfArea = sp.preProcess.two_column_data(z, smooth=6, minWave=2500, maxWave=10000

#Do scaling                                                                            
flux_pro = sfFlux/sfArea                                                                           

# Cutting the Spectra

Many of the tags use specific wavelength ranges of the spectrum rather than the whole thing and so we create multiple instances of the original spectrum cut at the corresponding wavelengths for each tag. 

In [ ]:
cuts = np.genfromtxt('/a_and_b.txt', dtype=int)

#Set wavelength indices
si_cuts = cuts[0]
he_cuts = cuts[1]
ca_cuts = cuts[2]
dp_cuts = cuts[3]
fe_cuts = cuts[4]
s_cuts = cuts[5]

#Create the cut spectra for the relevant tags
si_cut = flux_pro[:,Si_cuts[0]:Si_cuts[1]]
he_cut = flux_pro[:,He_cuts[0]:He_cuts[1]]
ca_cut = flux_pro[:,Ca_cuts[0]:Ca_cuts[1]]
dp_cut = flux_pro[:,dp_cuts[0]:dp_cuts[1]]
a5_cut = flux_pro[:,fe_cuts[0]:fe_cuts[1]]
a56_cut = flux_pro[:,s_cuts[0]:s_cuts[1]]

# Tagging

With spectra pre-processed and the necessary cuts made, we can now get the tag probabilities of the spectra and add them to an array ready to be given to the trained classifier.

In [ ]:
final = np.zeros([len(flux_pro),10])

#Get Hydrogen tag probabilities
H_result = log_reg_two(flux_pro, beta[0])
final[:,0] = H_result

#Get Silicon tag probabilities
Si_result = log_reg_two(si_cut, beta[1])
final[:,1] = Si_result

#Get Helium emission tag probabilities
He_emi_result = log_reg_two(he_cut, beta[2]

#Get Helium p cygni tag probabilities
He_cyg_result = log_reg_two(he_cut, beta[3])
final[:,3] = He_cyg_result

#Get Helium absorption tag probabilities
He_abs_result = log_reg_two(he_cut, beta[4])
final[:,4] = He_abs_result

#Get Hydrogen alpha tag probabilities
H_alp_result = log_reg_two(flux_pro, beta[5])
final[:,5] = H_alp_result

#Get Calcium tag probabilities
Ca_result = log_reg_two(ca_cut, beta[6])
final[:,6] = Ca_result

#Get Helium double peak tag probabilities
dp_result = log_reg_two(dp_cut, beta[7])
final[:,7] = dp_result

#Get Fe tag probabilities
fe_result = log_reg_two(fe_cut, beta[8]
final[:,8] = fe_result

#Get S tag probabilities
s_result = log_reg_two(s_cut, beta[9])
final[:,9] = s_result

# Classifying

We can now make our predictions for the class of the supernova by using the trained model. Since we are using softmax, we use 'np.argmax' to select the class with the highest probability, though one can see the probabilities of all the classes by printing 'class_prob'.

The predicted class is given a number, which corresponds to one of the 5 possible classes:

0 = Type Ia

1 = Type II

2 = Type Ib

3 = Type Ic

4 = Type IIb


In [ ]:
#Load in the trained model
model = keras.models.load_model('/Classifier_network2')

#Make classification prediction
class_prob = model.predict(final)
preds = np.argmax(class_prob2, axis=-1)
print("SN %s (with redshift %.3f) predicted class is %d with a %.3f probability " %  (name,z,preds,class_prob[preds))

# Tag Probabilities

One of the key features of STag is that all of the tags have probabilties, which can be accessed on demand.

In [ ]:
#Print the tag probabilities
print("%s - H: %.2f, Si: %.2f, Ca: %.2f, Fe: %.2f, S: %.2f, He emi: %.2f, He cyg: %.2f, He abs: %.2f, Ha: %.2f, He DP: %.2f" % (name,final[:,0],final[:,1],final[:,6],final[:,8],final[:,9],final[:,2],final[:,3],final[:,4],final[:,5],final[:,7]))

# Closing Remarks

One can use STag by following the steps outlined in this notebook, and with slight modifications one can adapt this code to run on multiple spectra rather than one at a time. 

Note that the classifying model used has only been trained on the 10 tags shown in this notebook, if one wishes to add additional tags then the model will need to be trained again. A more detailed description of how the tags have been made and how the model was built can be found in our paper: LINK GOES HERE.